In [ ]:
!pip install pytubefix
!sudo apt install ffmpeg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pytubefix import YouTube, Playlist
from pytubefix.cli import on_progress
import os
import subprocess

def download_from_url(url, path, resolution):
    faileds = []
    if not os.path.exists(path):
        os.makedirs(path)
    yt = YouTube(url, on_progress_callback = on_progress)
    if len((list(yt.streams.filter(file_extension='mp4', resolution=resolution)))):
        stream = yt.streams.filter(file_extension='mp4',resolution=resolution).desc().first()
    else:
        # the resolution is not available
        print("Resolution Not Available for this video, switching to the highest available quality")
        stream = yt.streams.filter(file_extension='mp4').order_by("resolution").desc().first()
    try:
        if stream.is_adaptive:
            raise Exception("The video is Adaptive")
        vid.download(path)
        print(f"Download of {yt.title} Completed")
    except:
        try:
            # Its DASH
            yt.streams.filter(file_extension='mp4').get_highest_resolution().download(filename=f"{yt.title}_with_sound.mp4")
            stream.download(filename=f"{yt.title}_without_sound.mp4")

            # Define the input and output files
            input_video_with_sound = f"./{yt.title}_with_sound.mp4"
            input_video_no_sound = f"./{yt.title}_without_sound.mp4"
            output_video = f"{path}/{yt.title}.mp4"

            # Use ffmpeg with GPU acceleration to combine the audio and video
            command = [
                "ffmpeg",
                "-i", input_video_no_sound,
                "-i", input_video_with_sound,
                "-c:v", "h264_nvenc",  # Use NVIDIA GPU for video encoding
                "-c:a", "aac",  # Use AAC codec for audio
                "-map", "0:v",  # Use video from the first input
                "-map", "1:a",  # Use audio from the second input
                output_video
            ]

            subprocess.run(command)

            os.remove(f"./{yt.title}_with_sound.mp4")
            os.remove(f"./{yt.title}_without_sound.mp4")
            print(f"Download of {yt.title} Completed")
        except Exception as e:
            print(f"Download of {yt.title} Failed")
            print(e)
            faileds.append(
                (video.title, video.watch_url)
            )
    if faileds:
        print("video(s) failed to download:")
        for failed in faileds:
            print(failed)

# Single Video

In [ ]:
#  Download a single clip using pytube

url = '<LINK>'
path = '/content/drive/MyDrive/youtube_dl'
resolution="720p"

download_from_url(url, path, resolution=resolution)

# Playlist

In [ ]:
################################################################################
URL = '<LINK>'
playlist = Playlist(URL)
# name of the new folder to be created
new_folder_name = 'new_folder'
resolution = "720p"
################################################################################
path = "/content/drive/MyDrive/youtube_dl/" + new_folder_name

for video in playlist.videos:
    print('downloading : {} with url : {}'.format(video.title, video.watch_url))
    url = video.watch_url
    download_from_url(url, path, resolution)

print("Playlist Download Completed")